In [23]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV 
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

In [2]:
path = r'C:\Users\Jchukwuedozi\Downloads\Data_for_UCI_named.csv'
df = pd.read_csv(path)

In [3]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   tau1    10000 non-null  float64
 1   tau2    10000 non-null  float64
 2   tau3    10000 non-null  float64
 3   tau4    10000 non-null  float64
 4   p1      10000 non-null  float64
 5   p2      10000 non-null  float64
 6   p3      10000 non-null  float64
 7   p4      10000 non-null  float64
 8   g1      10000 non-null  float64
 9   g2      10000 non-null  float64
 10  g3      10000 non-null  float64
 11  g4      10000 non-null  float64
 12  stab    10000 non-null  float64
 13  stabf   10000 non-null  object 
dtypes: float64(13), object(1)
memory usage: 1.1+ MB


In [5]:
len(df)

10000

In [6]:
df.drop(columns=['stab'], inplace=True)

In [7]:
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [8]:
X = df.iloc[:,:-1]
y = df.iloc[:, -1]

In [9]:
X.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923


In [10]:
print('Total length', len(df['stabf']))
print('length of stable', len(y[y == 'stable']))
print('length of unstable', len(y[y == 'unstable']))

Total length 10000
length of stable 3620
length of unstable 6380


In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [30]:
scaler = StandardScaler()
X_train_transform = scaler.fit_transform(X_train)
X_test_transform = scaler.transform(X_test)

In [31]:
forest = RandomForestClassifier(random_state=1)
forest.fit(X_train_transform, y_train)
X_pred = forest.predict(X_test_transform)
print('Classification Report')
print(classification_report(y_test, X_pred))
print('Confusion Matrix')
print(confusion_matrix(y_test, X_pred))
print(accuracy_score(y_test, X_pred))
# print(f'roc_auc_score {roc_auc_score(y_test, X_pred)}')

Classification Report
              precision    recall  f1-score   support

      stable       0.92      0.88      0.90       712
    unstable       0.93      0.96      0.95      1288

    accuracy                           0.93      2000
   macro avg       0.93      0.92      0.92      2000
weighted avg       0.93      0.93      0.93      2000

Confusion Matrix
[[ 625   87]
 [  55 1233]]
0.929


In [39]:
trees = ExtraTreesClassifier(random_state=1)
trees.fit(X_train_transform, y_train)
X_pred = trees.predict(X_test_transform)
print('Classification Report')
print(classification_report(y_test, X_pred))
print('Confusion Matrix')
print(confusion_matrix(y_test, X_pred))
print(accuracy_score(y_test, X_pred))
# print(f'roc_auc_score {roc_auc_score(y_test, X_pred)})

Classification Report
              precision    recall  f1-score   support

      stable       0.94      0.85      0.89       712
    unstable       0.92      0.97      0.95      1288

    accuracy                           0.93      2000
   macro avg       0.93      0.91      0.92      2000
weighted avg       0.93      0.93      0.93      2000

Confusion Matrix
[[ 606  106]
 [  38 1250]]
0.928


In [49]:
trees = ExtraTreesClassifier(n_estimators=1000, max_features=None, min_samples_leaf=8, min_samples_split=2, random_state=1)
trees.fit(X_train_transform, y_train)
X_pred = trees.predict(X_test_transform)
print('Classification Report')
print(classification_report(y_test, X_pred))
print('Confusion Matrix')
print(confusion_matrix(y_test, X_pred))
print(accuracy_score(y_test, X_pred))


Classification Report
              precision    recall  f1-score   support

      stable       0.92      0.87      0.89       712
    unstable       0.93      0.96      0.94      1288

    accuracy                           0.93      2000
   macro avg       0.93      0.91      0.92      2000
weighted avg       0.93      0.93      0.93      2000

Confusion Matrix
[[ 619   93]
 [  53 1235]]
0.927


In [45]:
xgboost = XGBClassifier(random_state=1)
xgboost.fit(X_train_transform, y_train)
X_pred = xgboost.predict(X_test_transform)
print('Classification Report')
print(classification_report(y_test, X_pred))
print('Confusion Matrix')
print(confusion_matrix(y_test, X_pred))
print(accuracy_score(y_test, X_pred))
# print(f'roc_auc_score {roc_auc_score(y_test, X_pred)})

c:\users\jchukwuedozi\desktop\pandas\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:11:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Classification Report
              precision    recall  f1-score   support

      stable       0.94      0.91      0.92       712
    unstable       0.95      0.97      0.96      1288

    accuracy                           0.95      2000
   macro avg       0.94      0.94      0.94      2000
weighted avg       0.95      0.95      0.95      2000

Confusion Matrix
[[ 648   64]
 [  45 1243]]
0.9455


In [36]:
lgbm = LGBMClassifier(random_state=1)
lgbm.fit(X_train_transform, y_train)
X_pred = lgbm.predict(X_test_transform)
print('Classification Report')
print(classification_report(y_test, X_pred))
print('Confusion Matrix')
print(confusion_matrix(y_test, X_pred, labels=['stable', 'unstable']))
print(accuracy_score(y_test, X_pred))
# print(f'roc_auc_score {roc_auc_score(y_test, X_pred)})

Classification Report
              precision    recall  f1-score   support

      stable       0.93      0.90      0.91       712
    unstable       0.95      0.96      0.95      1288

    accuracy                           0.94      2000
   macro avg       0.94      0.93      0.93      2000
weighted avg       0.94      0.94      0.94      2000

Confusion Matrix
[[ 641   71]
 [  50 1238]]
0.9395


In [47]:
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]
param_grid = {'xtrees__n_estimators': n_estimators,
             'xtrees__min_samples_split': min_samples_split,
             'xtrees__min_samples_leaf': min_samples_leaf,
             'xtrees__max_features': max_features}
pipe = Pipeline(steps=[('scaler', StandardScaler()), ('xtrees', ExtraTreesClassifier())])
random_search = RandomizedSearchCV(pipe, param_grid, cv=5, scoring='accuracy', random_state=1, verbose=1, n_jobs=-1, n_iter=10)
random_search.fit(X_train_transform, y_train)
print('Best Parameters')
print(random_search.best_params_)
print('Best Score')
print(random_search.best_score_)
print('Best Estimator')
print(random_search.best_estimator_)
X_pred = random_search.predict(X_test_transform)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Parameters
{'xtrees__n_estimators': 1000, 'xtrees__min_samples_split': 2, 'xtrees__min_samples_leaf': 8, 'xtrees__max_features': None}
Best Score
0.924
Best Estimator
Pipeline(steps=[('scaler', StandardScaler()),
                ('xtrees',
                 ExtraTreesClassifier(max_features=None, min_samples_leaf=8,
                                      n_estimators=1000))])
